# Тестирование модели, обученной ранее, на новых текстах. 

Задача: предсказать категорию (тему) сбора по тексту. Для теста вручную собраны 10 текстов, которые не участвовали в обучении и оценки при обучении. Тексты взяты с новых площадок: Boomstarter и Dobro.mail.ru (датасет для обучения состоял из текстов с Planeta.ru). 

In [3]:
%%time
from prep import Prep #вызываем сохраненный класс (хранится в файле prep.py)

                                        precision    recall  f1-score   support

     активизм просвещение профилактика       0.36      0.19      0.25        21
                    бездомные и кризис       0.80      0.89      0.84         9
       взрослые лечение и реабилитация       0.67      0.33      0.44        18
           дети лечение и реабилитация       0.84      0.92      0.88       134
                              животные       0.92      1.00      0.96        61
                           заключенные       0.00      0.00      0.00         1
                   малоимущие бедность       0.41      0.53      0.46        17
            мечты, подарки и праздники       0.61      0.48      0.54        23
                наука история культура       0.00      0.00      0.00         3
                      паллиатив и уход       0.85      0.79      0.81        14
                    пожилые и ветераны       0.89      0.73      0.80        11
         развитие нко и инфраструктура 

In [8]:
from joblib import load # для загрузки модели
import pandas as pd

In [9]:
clf = load('pipe.joblib') # грузим сохраненную модель

Сначала попробуем протестировать на произвольных строках

In [10]:
clf.predict(pd.core.series.Series(["Мама мыла раму"]))

array(['дети_лечение_реабилитация'], dtype=object)

In [11]:
clf.predict(pd.core.series.Series(['Человек и кошка плачут у окошка']))

array(['дети_лечение_реабилитация'], dtype=object)

In [12]:
clf.predict(pd.core.series.Series(['Дай, Джим, на счастье лапу мне. Такую лапу не видал я сроду']))

array(['дети_лечение_реабилитация'], dtype=object)

In [13]:
clf.predict(pd.core.series.Series(['А лисички взяли спички, к морю синему пошли, море синее сожгли']))

array(['дети_лечение_реабилитация'], dtype=object)

In [14]:
clf.predict(pd.core.series.Series(['бездомный бездомный улица дить на улице зима еда ночрег']))

array(['дети_лечение_реабилитация'], dtype=object)

\
\
Итог: результат плохой. Модель все время предсказывает мажоритарный класс (в датасете, на котором проводилось обучение, из 16 категорий в этой категории 25% текстов). Проверим на настоящих текстах.

In [73]:
boom = pd.read_csv('/Users/liza/PycharmProjects/Planeta_project/Dobro_boom.tsv', sep ="\t") # тексты с Boomstarter и Dobro.mail.ru

In [74]:
boom

,text,link
0,Лариса Ермошина живет в городе Донской Тульско...,https://dobro.mail.ru/projects/zhizn-dlya-lari...
1,"Удар был такой силы, что Шанс встать уже не см...",https://dobro.mail.ru/projects/rejdom-vsem-bud...
2,"«Доброе утро, Любовь Алексеевна! Вот молоко, т...",https://dobro.mail.ru/projects/ne-bojsya-ya-s-...
3,«Человек-радио» – так в шутку называют Сашу в ...,https://dobro.mail.ru/projects/podarit-dyihani...
4,"«Почему листья зеленые?», «Что такое железо?»,...",https://dobro.mail.ru/projects/preodolet-izoly...
5,В марте 2017 года студент - медик Евгений Косо...,https://boomstarter.ru/projects/dm/drugaya_med...
6,2020 год Указом Президента РФ объявлен Годом п...,https://boomstarter.ru/projects/Serdze88/roves...
7,"Меня зовут Максим Батырев, я автор книг-бестсе...",https://boomstarter.ru/projects/591685/memoria...
8,"Всем привет! Мы - движение ""Молодёжь за мир"".\...",https://boomstarter.ru/projects/914463/obedy_d...
9,"Здравствуйте, я – Сергей Богдановский, руковож...",https://boomstarter.ru/projects/taganai89/sozd...


In [75]:
boom.shape

(10, 2)

In [78]:
# просим модель определить категории для всех текстов в новом датасете

cats = clf.predict(boom['text'])
cats

array(['взрослые_лечение_реабилитация', 'животные', 'пожилые_ветераны',
       'дети_лечение_реабилитация', 'социализация_возможности',
       'бездомные_кризис', 'пожилые_ветераны',
       'дети_лечение_реабилитация', 'бездомные_кризис',
       'социализация_возможности'], dtype=object)

In [79]:
# делаем так, чтобы удомбно было проверить глазами, что как предсказалось

lcats = list(cats) # превращаем numpy.array которую вернула модель, в список, чтобы итерироваться ниже
ltexts = list(boom['text']) # тексты из датасета тоже из series превращаем в список

for x, y in zip(ls, b): # итерируемся по ним параллельно
    print(f"Category: {x}\n\n{y}\n\n***\n")

# результат: 9 из 10, подробнее - ниже

Category: взрослые_лечение_реабилитация

Лариса Ермошина живет в городе Донской Тульской области и работает на обувной фабрике. Муж, дочь, работа, дом — обычная жизнь.

Но с некоторых пор Лариса почувствовала, будто что-то с нею не так. Что именно не так? Нельзя было сказать внятно.

Что-то болело? Ну, иногда что-то болело. В сорок восемь лет уже часто что-то болит: то голова, то спина. Но болело не сильно, а чувство «что-то не так» было сильное.

Лариса пошла к врачу. Не сформулировала внятно никаких жалоб, и доктор внятно ничего не сказал. Пошла к другому врачу. На что жалуетесь? Головокружения. У всех головокружения. Но степень своих головокружений Лариса не могла передать.

Близкие ей сочувствовали. Друзья. Сослуживцы. Коллега, живущая в соседнем доме, однажды предложила Ларисе возить ее на работу. Прежде Лариса ездила на общественном транспорте, а у коллеги была машина. Лариса согласилась.

Приезжала на работу, сидела на рабочем месте и не могла приклеить ни одной подошвы ни к одн

# Выводы

1. Из 10 текстов - разнообразных по тематике и с площадок, с которыми модель не знакома, 9 текстов определены верно. Одна ошибка получилась с текстом (Педагогическая поэма, Макаренко), в котором действительно смешанная лексика. Модель здесь определила "лечение детей", хотя тема ближе к культуре. 

2. Видно, что тексты, несопоставимые по объему, то есть по лексической насыщенности, с теми, на которых модель училась, она не может обработать качественно (см. выше тесты на отд. предложениях). В таком случае, модель всегда предсказывает мажоритарный класс. В дальнейшем можно повторить обучение - теперь с примененеим методик работы с несбалансированными данными. 